In [1]:
'''
@Project ：Diffusion Reaction PDE
@File    ：Diffusion_Reaction.py
@IDE     ：VS Code
@Author  ：Xuepeng Cheng
@Date    ：2024年8月13日 
'''
import torch

torch.set_default_dtype(torch.float32)
import numpy as np
import pandas as pd
import os
import math

import matplotlib.pyplot as plt
from matplotlib import cm

from DR_KAN import Diffusion_Reaction,Sampler


if __name__ == '__main__':
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    
    """Hard Code Part"""
    L = 600 ##nm
    h = 100 ##nm
    geom = [0,L]

    anneal_time= 60 ##min
    TimeDomain = [0,anneal_time]
        
    D_A = 360  #6nm^2/s 360nm^2/min
    k11,k12,k21 = 3.6e-3,3.6e-3,0

    N_SA,N_SB,N_SC,N_SAB,N_SABB,N_SAAB = 90,48,90,90,90,90

    lam_threshold = 1e6

    sampler = Sampler(geom,TimeDomain,name='coordinates')
    results = []

    for depth in [3]:  ##[4,5,6,7,8]
        for widths in [20]: ##[56,64,72]
            layer = [2] + [widths]*depth + [6]
            layer_str = f'{depth}x{widths}'
            for mode in ['ChebyKAN','KAN']:  # ['PINN','IA_PINN'] ['IAW_PINN','I_PINN]
                torch.cuda.empty_cache()
                
                model = Diffusion_Reaction(layer, sampler,
                                D_A, k11, k12, k21,
                                N_SA,N_SB,N_SC,N_SAB,N_SABB,N_SAAB,
                                h,L,anneal_time,
                                mode, lam_threshold)
                model.train(nIter = 6000,num_interior = 100,num_boundary =2500, num_initial =10000,additional_points=0)              
                
                """Validation Part"""
                x = np.linspace(0,1,100)
                t = np.linspace(0,1,100)
                ms_x,ms_t = np.meshgrid(x,t)

                x = np.ravel(ms_x).reshape(-1, 1)
                t = np.ravel(ms_t).reshape(-1, 1)
                pt_x = torch.from_numpy(x).float().requires_grad_(True).to(device)
                pt_t = torch.from_numpy(t).float().requires_grad_(True).to(device)
                X_star = torch.cat([pt_x,pt_t],1).to(device)

                result = model.predict_u(X_star).data.cpu().numpy()
                c_a,c_bc,c_c,c_ab,c_abb,c_aab = result[:,0],result[:,1],result[:,2],result[:,3],result[:,4],result[:,5]
                c_a,c_bc,c_c,c_ab,c_abb,c_aab = [np.where((tensor < 0),np.array(0),tensor) for tensor in [c_a,c_bc,c_c,c_ab,c_abb,c_aab]]

                data_dict_raw = {
                    "c_a": c_a,
                    "c_bc": c_bc,
                    "c_c": c_c,
                    "c_ab": c_ab,
                    "c_abb": c_abb,
                    "c_aab": c_aab
                }

                for name, concentration in data_dict_raw.items():
                    # Convert the array to a DataFrame
                    concentration = concentration.reshape(100, 100)

                    df = pd.DataFrame(concentration)
                    # print(df)
                    df = df.transpose() 

                    new_df = pd.DataFrame(np.nan, index=range(101), columns=range(101))
                    new_df.iloc[0, 1:] = np.linspace(0,60,100)
                    new_df.iloc[1:,0] = np.linspace(0,600,100)

                    new_df.iloc[1:, 1:] = df.values
                    folder_path = f"./Results/reaction t = 60/csv/RAW/{layer_str}_{mode}/"
                    os.makedirs(folder_path, exist_ok=True)
                    new_df.to_csv(os.path.join(folder_path, f"{name}.csv"), index=False, header=False)

                c_a,c_bc,c_c,c_ab,c_abb,c_aab = c_a*N_SA, c_bc*N_SB, c_c*N_SC, c_ab*N_SAB, c_abb*N_SABB, c_aab*0
                
                c_a=c_a.reshape(100,100)
                c_bc=c_bc.reshape(100,100)
                c_c=c_c.reshape(100,100)
                c_ab=c_ab.reshape(100,100)
                c_abb=c_abb.reshape(100,100)
                c_aab=c_aab.reshape(100,100)

                data_dict = {
                    "c_a": c_a,
                    "c_bc": c_bc,
                    "c_c": c_c,
                    "c_ab": c_ab,
                    "c_abb": c_abb,
                    "c_aab": c_aab
                }
                for name, concentration in data_dict.items():
                    df = pd.DataFrame(concentration)
                    # print(df)
                    df = df.transpose() 

                    new_df = pd.DataFrame(np.nan, index=range(101), columns=range(101))
                    new_df.iloc[0, 1:] = np.linspace(0,60,100)
                    new_df.iloc[1:,0] = np.linspace(0,600,100)

                    new_df.iloc[1:, 1:] = df.values
                    folder_path = f"./Results/reaction t = 60/csv/Original/{layer_str}_{mode}/"
                    os.makedirs(folder_path, exist_ok=True)
                    new_df.to_csv(os.path.join(folder_path, f"{name}.csv"), index=False, header=False)

                sum = c_a[99,:] + c_bc[99,:] + c_c[99,:] + c_ab[99,:] + c_abb[99,:] + c_aab[99,:]
                
                re_a = c_a[99,:] / sum * 100
                re_bc = c_bc[99,:] / sum * 100 /2
                re_c = c_c[99,:] / sum * 100
                re_ab = c_ab[99,:] / sum * 100
                re_abb = c_abb[99,:] / sum * 100
                re_aab = c_aab[99,:] / sum * 100

                loss_pde = model.loss_pde_log
                loss_bc = model.loss_bc_log
                loss_ic = model.loss_ic_log
                loss_total = model.loss_total_log

                component_x = np.linspace(0, 600, 100)

                fig_1,ax = plt.subplots(1,2,figsize=(14,6))
                ax[0].plot(component_x, re_a, label=r'Concentration of Ni', color=(97/255,108/255,140/255))
                ax[0].plot(component_x, re_bc, label=r'Concentration of SiC', color=(86/140,140/255,135/255))
                ax[0].plot(component_x, re_c, label=r'Concentration of C', color=(178/255,213/255,155/255))
                ax[0].plot(component_x, re_ab, label=r'Concentration of NiSi', color=(242/255,222/255,121/255))
                ax[0].plot(component_x, re_abb, label=r'Concentration of NiSi2', color=(217/255,95/255,24/255))
                ax[0].set_title('Relative Concentration,time = 60')
                ax[0].set_xlabel('x')
                ax[0].set_ylabel('Concentration, %')
                ax[0].set_xlim([0, 600])
                ax[0].set_ylim([0, 110])
                ax[0].legend()

                ax[1].plot(loss_total, label='$\mathcal{L}_{totall}$')
                ax[1].plot(loss_pde, label='$\mathcal{L}_{pde}$')
                ax[1].plot(loss_bc, label='$\mathcal{L}_{bc}$')
                ax[1].plot(loss_ic, label='$\mathcal{L}_{ic}$')
                ax[1].set_yscale('log')
                ax[1].set_xlabel('iterations')
                ax[1].set_ylabel('Loss')
                ax[1].legend()

                save_path = f'./Results/reaction t = 60/'
                file_name = f'{layer_str}_{mode}_loss.png'

                if not os.path.exists(save_path):
                    os.makedirs(save_path)

                plt.tight_layout()
                plt.savefig(os.path.join(save_path, file_name))
                plt.close()

                relative_u1 = model.log_relative_u1
                relative_u2 = model.log_relative_u2
                relative_u3 = model.log_relative_u3
                relative_u4 = model.log_relative_u4
                relative_u5 = model.log_relative_u5
                fig_2,ax = plt.subplots(1,5,figsize=(22,4))
                ax = ax.flatten()

                def annotate_last_point(ax, data, label):
                    x = len(data) - 1
                    y = data[-1]
                    ax.plot(x, y, 'ro',markersize = 2)  
                    ax.annotate(f'{y:.2e}', xy=(x, y), xytext=(x, y*1.1))
                    
                ax[0].plot(relative_u1)
                ax[0].set_title('Relative Error of Ni')
                ax[0].set_yscale('log')
                annotate_last_point(ax[0], relative_u1, 'Ni')

                ax[1].plot(relative_u2)
                ax[1].set_title('Relative Error of SiC')
                ax[1].set_yscale('log')
                annotate_last_point(ax[1], relative_u2, 'SiC')

                ax[2].plot(relative_u3)
                ax[2].set_title('Relative Error of C')
                ax[2].set_yscale('log')
                annotate_last_point(ax[2], relative_u3, 'C')

                ax[3].plot(relative_u4)
                ax[3].set_title('Relative Error of NiSi')
                ax[3].set_yscale('log')
                annotate_last_point(ax[3], relative_u4, 'NiSi')

                ax[4].plot(relative_u5)
                ax[4].set_title('Relative Error of $NiSi_2$')
                ax[4].set_yscale('log')
                annotate_last_point(ax[4], relative_u5, '$NiSi2$')

                save_path = f'./Results/reaction t = 60/'
                file_name = f'{layer_str}_{mode}_relative_Error.png'

                if not os.path.exists(save_path):                                       
                    os.makedirs(save_path)

                plt.tight_layout()
                plt.savefig(os.path.join(save_path, file_name))
                plt.close()


Current device: cuda:0

 model: ChebyKAN, layer: [2, 20, 20, 20, 6]


  2%|▉                                                             | 95/6000 [00:29<30:27,  3.23it/s, Loss=3.008e-01, loss_pde=8.814e-02, loss_bc=1.408e-02, loss_ic=1.986e-01, grouped_ic=1.97e-01,1.32e-03, lr=1.00e-02, D*=5.98e-02,7.55e-03]


KeyboardInterrupt: 